In [27]:
import numpy as np
import pandas as pd
import sklearn.metrics 

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection

import xgboost as xgb
from xgboost import XGBClassifier

from numpy import sort
import scipy.stats as stats

In [28]:
df_train_mean_encoding = pd.read_csv('../Feature_Encoding/data/train_mean_encoding.csv')
df_test_mean_encoding = pd.read_csv('../Feature_Encoding/data/test_mean_encoding.csv')
df_train_binary_encoding = pd.read_csv('../Feature_Encoding/data/train_binary_encoding.csv')
df_test_binary_encoding = pd.read_csv('../Feature_Encoding/data/test_binary_encoding.csv')
train = pd.read_csv('../Feature_Engineering/data/other-cleaned_train.csv')

In [29]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [34]:
#Realiza busqueda completa combinando los parametros
def xgboost(x_train, y_train, x_validation, y_validation):
    xgb_classifier = XGBClassifier()
    params_xgb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.05, 0.1], 'gamma': [0, 1, 5]}    
    xgb_gs = GridSearchCV(xgb_classifier, params_xgb, cv=5)
    xgb_gs.fit(x_train, y_train)
    xgb_best = xgb_gs.best_estimator_
    print(xgb_gs.best_params_)
    print('xgb: {}'.format(xgb_best.score(x_validation, y_validation)))
    return xgb_best


#Realiza busqueda random dentro de los parametros validos
def xgboost2(x_train, y_train, x_validation, y_validation):
    xgb_classifier = XGBClassifier()
    params_xgb_2={
        'n_estimators':stats.randint(10,500),'learning_rate':stats.uniform(0.01,0.3),
        'subsample':stats.uniform(0.3,0.7),'min_child_weight':[1,5,10],
        'max_depth':[3,10,6],'gamma':stats.randint(0,10),'colsample_bytree':stats.uniform(0.,0.6)
    }
    
    xgb_rs = RandomizedSearchCV(xgb.XGBClassifier(n_jobs=-1),
                          param_distributions=params_xgb_2,
                          cv=2,
                          scoring='neg_log_loss',
                          verbose=1,
                          n_iter=200)
    
    xgb_rs.fit(x_train, y_train)
    xgb_best = xgb_rs.best_estimator_
    print(xgb_rs.best_params_)
    print('xgb: {}'.format(xgb_best.score(x_validation, y_validation)))
    return xgb_best

def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = log_loss(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [31]:
y = train.Target
x_train_mean_encoding, x_validation_mean_encoding, y_train_mean_encoding, y_validation_mean_encoding = train_test_split(df_train_mean_encoding, y, test_size=0.3, stratify=y)
x_train_binary_encoding, x_validation_binary_encoding, y_train_binary_encoding, y_validation_binary_encoding = train_test_split(df_train_binary_encoding, y, test_size=0.3, stratify=y)

## Mean encoding

In [32]:
xgboost_mean_encoding = xgboost(x_train_mean_encoding, y_train_mean_encoding, x_validation_mean_encoding, y_validation_mean_encoding)
test_model(xgboost_mean_encoding,x_validation_mean_encoding,y_validation_mean_encoding)
cross_val(xgboost_mean_encoding, x_train_mean_encoding, y_train_mean_encoding)

{'gamma': 0, 'learning_rate': 0.1, 'n_estimators': 200}
xgb: 0.8923988153998026
Accuracy: 89.24%, Logloss: 0.24
0.8937210935939216


In [35]:
xgboost_mean_encoding = xgboost2(x_train_mean_encoding, y_train_mean_encoding, x_validation_mean_encoding, y_validation_mean_encoding)
test_model(xgboost_mean_encoding,x_validation_mean_encoding,y_validation_mean_encoding)
cross_val(xgboost_mean_encoding, x_train_mean_encoding, y_train_mean_encoding)

Fitting 2 folds for each of 200 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 22.6min finished


{'colsample_bytree': 0.28630949913269416, 'gamma': 0, 'learning_rate': 0.0906709697875075, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 210, 'subsample': 0.9239971976963097}
xgb: 0.9342546890424481
Accuracy: 93.43%, Logloss: 0.16
0.9273143921411094


## Binary encoding

In [36]:
xgboost_binary_encoding = xgboost(x_train_binary_encoding, y_train_binary_encoding, x_validation_binary_encoding, y_validation_binary_encoding)
test_model(xgboost_binary_encoding,x_validation_binary_encoding,y_validation_binary_encoding)
cross_val(xgboost_binary_encoding, x_train_binary_encoding, y_train_binary_encoding)

{'gamma': 1, 'learning_rate': 0.1, 'n_estimators': 200}
xgb: 0.8756169792694966
Accuracy: 87.56%, Logloss: 0.29
0.8710433981867954


In [38]:
xgboost_binary_encoding = xgboost2(x_train_binary_encoding, y_train_binary_encoding, x_validation_binary_encoding, y_validation_binary_encoding)
test_model(xgboost_binary_encoding,x_validation_binary_encoding,y_validation_binary_encoding)
cross_val(xgboost_binary_encoding, x_train_binary_encoding, y_train_binary_encoding)

Fitting 2 folds for each of 200 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 55.2min finished


{'colsample_bytree': 0.20900672188191186, 'gamma': 1, 'learning_rate': 0.10874310026757804, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 487, 'subsample': 0.8846703276847963}
xgb: 0.9251727541954591
Accuracy: 92.52%, Logloss: 0.19
0.9207137246309252


In [43]:
y_pred = xgboost_mean_encoding.predict_proba(df_test_mean_encoding)[:,1]
submission_xgboost = pd.DataFrame(data={'Opportunity_ID':df_test_mean_encoding['Opportunity_ID'], 'Target': y_pred})
submission_xgboost = submission_xgboost.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_xgboost.to_csv('submits/mean_xgboost.csv', index=False)

In [42]:
y_pred = xgboost_binary_encoding.predict_proba(df_test_binary_encoding)[:,1]
submission_xgboost = pd.DataFrame(data={'Opportunity_ID':df_test_binary_encoding['Opportunity_ID'], 'Target': y_pred})
submission_xgboost = submission_xgboost.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_xgboost.to_csv('submits/binary_xgboost.csv', index=False)